# Demo for model comparison with SARIMA model

This notebook demo the WIP workflow of incorporating real ML Model to model comparison table. 


## Fetch data
The below cell grabs data from blob and save to local directory. 
Only need to be runned for once.

It will save the data to `downloads/`, which is already git-ignored for this repo.

Please make sure you fill in the blob account name and access key in `config.ini` . Also, run `azure login`.

In [1]:
from pipeline.utils import connect_spark_to_blob_storage, load_latest_blob_to_pyspark


connect_spark_to_blob_storage()

table_name = 'SpendDataFilledMissingMonth'
sdf = load_latest_blob_to_pyspark(table_name=table_name)
sdf.write.parquet(f'downloads/{table_name}.parquet', mode='overwrite')

:: loading settings :: url = jar:file:/Users/joe.fong/deliveries/ccs/ccs-spend-forecasting-app/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/joe.fong/.ivy2/cache
The jars for the packages stored in: /Users/joe.fong/.ivy2/jars
org.apache.hadoop#hadoop-azure added as a dependency
com.microsoft.sqlserver#mssql-jdbc added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b1e68b0b-fc8e-4fd1-987b-5b5a93eaefce;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-azure;3.3.4 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found com.microsoft.azure#azure-storage;7.0.1 in central
	found com.fasterxml.jackson.core#jackson-core;2.12.7 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found com.microsoft.azure#azure-keyvault-core;1.0.0 in central
	found com.google.guava#guava;27.0-jre in central
	found com.g

22/12/14 17:51:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/14 17:52:01 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-azure-file-system.properties,hadoop-metrics2.properties


## Prepare input data

In [2]:
import pandas as pd


table_name = 'SpendDataFilledMissingMonth'

# read data from local file to pandas
df = pd.read_parquet(f'downloads/{table_name}.parquet')

# sum up the spend data by month, so that for each combination, only one row for one month 
df = df.groupby(["SpendMonth", "Category", "MarketSector"], as_index=False).agg(
        {"EvidencedSpend": "sum"}
    )

# select only a few combinations to make comparison for
# if find_hyperparameters is set to True, then it every combination will run a search for best parameter, which take around 10 mins.
category_list = ["Workforce Health & Education", "Network Services"]
market_sector_list = ["Health", "Education"]

input_df = df[(df['Category'].isin(category_list)) & (df['MarketSector'].isin(market_sector_list))]


## Run the models and create comparison table

The below cell creates a SARIMA model and a mock model, then create a comparison table for them.


For the SARIMA model, currently a hardcoded set of hyperparameter (~= variable) is used, so some combinations will perform quite poorly.

The reason of hardcoding here is because searching for a set of good params take around 10 minutes time per combination.

By changing the flag `find_hyperparameters` to True in `pipeline/jobs/sarima_model.py`, you can enforce it to ignore the hardcoded params and run a search for hyperparameter.

TODO: Find a better way to store and retreive params for each Category/MarketSector


In [3]:
from pipeline.jobs.sarima_model import SarimaModel
from pipeline.jobs.mock_forecast import create_mock_model
from pipeline.jobs.models_comparison import create_models_comparison
import datetime


# Compare the performance of SARIMA model with a random mock model.

sarima = SarimaModel()

models = [
    sarima, 
    create_mock_model(name='MockModel', randomness=0.3)
]


comparison_table = create_models_comparison(
    input_df=input_df,
    train_ratio=0.9,
    models=models
)

comparison_table


22/12/14 17:52:18   DEBUG    sarima_model. 87: Generating forecast for ('Network Services', 'Education')...
/Users/joe.fong/deliveries/ccs/ccs-spend-forecasting-app/.venv/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/joe.fong/deliveries/ccs/ccs-spend-forecasting-app/.venv/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
22/12/14 17:52:21   DEBUG    sarima_model. 87: Generating forecast for ('Network Services', 'Health')...
/Users/joe.fong/deliveries/ccs/ccs-spend-forecasting-app/.venv/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starti

,Category,MarketSector,SpendMonth,EvidencedSpend,SARIMA Forecast,SARIMA Error %,SARIMA MAPE,MockModel Forecast,MockModel Error %,MockModel MAPE,Model Suggested
2,Network Services,Education,2021-12-01,1.652035e+06,2.445616e+06,0.480366,0.318745,2.235336e+06,0.353080,0.260082,MockModel
7,Network Services,Education,2022-01-01,2.614018e+06,2.725858e+06,0.042785,0.318745,1.971957e+06,0.245622,0.260082,MockModel
9,Network Services,Education,2022-02-01,2.046709e+06,3.187674e+06,0.557463,0.318745,2.874033e+06,0.404222,0.260082,MockModel
12,Network Services,Education,2022-03-01,3.468452e+06,5.779140e+06,0.666201,0.318745,2.008640e+06,0.420883,0.260082,MockModel
16,Network Services,Education,2022-04-01,2.473907e+06,3.421351e+06,0.382975,0.318745,1.877386e+06,0.241125,0.260082,MockModel
21,Network Services,Education,2022-05-01,2.514158e+06,2.629629e+06,0.045928,0.318745,3.143340e+06,0.250256,0.260082,MockModel
27,Network Services,Education,2022-06-01,3.259681e+06,2.659963e+06,0.183981,0.318745,2.374993e+06,0.271403,0.260082,MockModel
31,Network Services,Education,2022-07-01,2.661466e+06,2.859103e+06,0.074259,0.318745,2.996324e+06,0.125817,0.260082,MockModel
34,Network Services,Education,2022-08-01,2.547210e+06,3.169055e+06,0.244128,0.318745,3.166158e+06,0.242991,0.260082,MockModel
38,Network Services,Education,2022-09-01,2.456073e+06,3.925289e+06,0.598198,0.318745,3.035370e+06,0.235863,0.260082,MockModel
